In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
print(tf.__version__)
print(np.__version__)

2.3.0
1.19.2


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/cjfghk5697/forecastsolar/main/csv%20file/energy.csv") # 발전소별 발전량

In [4]:
date_time = pd.to_datetime(df.pop('time'), format='%Y.%m.%d %H:%M:%S')

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0
